# Model Data

In [1]:
%reload_ext autoreload
%autoreload 2

In [3]:
#export
from pymongo import *
from exp.nb_00 import *
from exp.nb_01 import *
from exp.nb_02 import *

In [4]:
client = MongoClient(host='localhost', port=27017)

# lblnew-bestfit-sw model data  
Here we try to define a class that will represent data associated with lblnew-bestfit-sw.

## lblnew-bestfit-lw model data's structure

We review lblnew-bestfit-lw model data's structure here, and will try to use as similar a structure for lblnew-bestfit-sw.

In [20]:
from climatools.dataset import *

In [21]:
client.database_names()

['admin', 'cliradnew', 'config', 'lblnew', 'local']

In [22]:
client.lblnew.collection_names(), client.cliradnew.collection_names()

(['overlap_lw', 'bestfit_lw'], ['lw'])

In [23]:
doc = client.lblnew.bestfit_lw.find_one()
md = LBLnewBestfitModelData.from_mongodoc(doc)

In [24]:
vars(md).keys()

dict_keys(['dgdgs', 'abscom', 'param', 'wgt_flux', 'wgt_cool', 'crd_flux', 'crd_cool'])

In [25]:
print('lblnew-bestfit-lw output dimensions')
md.wgt_flux.dims, md.wgt_cool.dims, md.crd_flux.dims, md.crd_cool.dims

lblnew-bestfit-lw output dimensions


(Frozen(SortedKeysDict({'pressure': 76, 'igg': 10, 'g': 9})),
 Frozen(SortedKeysDict({'pressure': 75, 'igg': 10, 'g': 9})),
 Frozen(SortedKeysDict({'pressure': 76, 'g': 9})),
 Frozen(SortedKeysDict({'pressure': 75, 'g': 9})))

In [26]:
print('Flux data')
print(40 * '-')
print('wgt')
print(md.wgt_flux.data_vars)
print(40 * '-')
print('crd')
print(md.crd_flux.data_vars)

Flux data
----------------------------------------
wgt
Data variables:
    flug     (pressure, igg, g) float64 -0.01002 -0.03926 ... -8.293 -13.63
    fldg     (pressure, igg, g) float64 0.0 0.0 0.0 0.0 ... 9.244 4.633 1.224
    fnetg    (pressure, igg, g) float64 -0.01002 -0.03926 ... -3.66 -12.41
----------------------------------------
crd
Data variables:
    flug     (pressure, g) float64 -0.01029 -0.03859 -0.164 ... -8.268 -13.4
    fldg     (pressure, g) float64 0.0 0.0 0.0 0.0 ... 2.328 9.234 4.48 1.215
    fnetg    (pressure, g) float64 -0.01029 -0.03859 -0.164 ... -3.788 -12.19


In [27]:
print('Cool data')
print(40 * '-')
print('wgt')
print(md.wgt_cool.data_vars)
print(40 * '-')
print('crd')
print(md.crd_cool.data_vars)

Cool data
----------------------------------------
wgt
Data variables:
    coolrg   (pressure, igg, g) float64 0.8167 -0.004995 ... 0.07284 0.03006
----------------------------------------
crd
Data variables:
    coolrg   (pressure, g) float64 0.8949 0.01439 -0.004269 ... 0.06004 0.02779


## lblnew-bestfit-sw's fort* output files  
How the output data are organised in the various "fort" files in the current lblnew-bestfit-sw.  At the moment the output fort.\* files give absolutly no indication of the dimensions, so we will need to refer the the ones already figured out in "analysis_-_lblnew-bestfit-sw.ipynb".

In [135]:
param0 = LBLnewBestfitSWParam(band=9, vmin=8200, vmax=14290, nv=10000, dv=0.001,
                     molecule='h2o', conc='atmpro',
                     ref_pts=[(30, 250), (300, 250)], ng_refs=[3, 9], ng_adju=[-3, 0],
                     wgt=[(.95, .9, .6, .5), (.55, .55, .55, .65, .65, .7, .8, .9)],
                     cosz=.2588, rsfc=0,
                     klin=3e-25,
                     option_k_lookup=0,
                     atmpro='trp',
                     commitnumber='79f2292')

In [136]:
path = PATH_ANALYSIS/'/'.join(sorted(f'{n}_{v}' for n, v in vars(param0).items()))
runner = LBLnewBestfitSWRun(path=path, param=param0)

In [137]:
fids = sorted([401, 400, 10, 402, 403, 9, ])
fids

[9, 10, 400, 401, 402, 403]

In [151]:
heat = pd.read_csv(runner.path/'fort.401', header=None, sep=r'\s+')
heat.set_index(0, inplace=True)
heat.index.names = ['pressure']
heat.columns.names = ['g']

print('crd heating rate for each g.')
xr.Dataset.from_dataframe(pd.DataFrame(heat.stack()))
#xr.Dataset.from_dataframe(heat.stack())

crd heating rate for each g.


<xarray.Dataset>
Dimensions:   (g: 12, pressure: 75)
Coordinates:
  * pressure  (pressure) float64 0.0003 0.0008 0.0011 ... 953.6 977.3 1.001e+03
  * g         (g) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    0         (pressure, g) float64 0.002636 0.001432 ... 0.1543 0.1532

In [150]:
heatg = pd.read_csv(runner.path/'fort.400', header=None, sep=r'\s+')
heatg.set_index(0, inplace=True)
heatg.index.names = ['pressure']
heatg.columns.names = ['g']

print('wgt heating rate for each g')
xr.Dataset.from_dataframe(pd.DataFrame(heatg.stack()))

wgt heating rate for each g


<xarray.Dataset>
Dimensions:   (g: 12, pressure: 75)
Coordinates:
  * pressure  (pressure) float64 0.0003 0.0008 0.0011 ... 953.6 977.3 1.001e+03
  * g         (g) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    0         (pressure, g) float64 0.002602 0.001383 ... 0.1645 0.1584

In [148]:
heat = pd.read_csv(runner.path/'fort.10', sep=r'\s+', header=None)
heat.set_index(0, inplace=True)
heat.index.names = ['pressure']
heat.columns = ['CRD', 'WGT']

print('bands-total heating rate')
xr.Dataset.from_dataframe(heat)

bands-total heating rate


<xarray.Dataset>
Dimensions:   (pressure: 75)
Coordinates:
  * pressure  (pressure) float64 0.0003 0.0008 0.0011 ... 953.6 977.3 1.001e+03
Data variables:
    CRD       (pressure) float64 0.0052 0.0052 0.0052 ... 0.4694 0.4592 0.451
    WGT       (pressure) float64 0.00491 0.00491 0.00491 ... 0.4889 0.481 0.4755

In [109]:
def load_flux(fpath=None, name=None):
    '''
    Return flux for each g-group, either calculated by line-by-line (fort.403), 
    or by clirad-sw (fort.402), in a xarray.DataArray.
    '''
    df = pd.read_csv(fpath, sep=r'\s+', header=None)
    df.set_index(0, inplace=True)
    df.index.names = ['pressure']
    df.columns.names = ['g']
    da = xr.DataArray(df)
    da.name = name
    return da

In [168]:
dwgt = load_flux(fpath=runner.path/'fort.402', name='WGT')
dcrd = load_flux(fpath=runner.path/'fort.403', name='CRD')
ds = xr.Dataset()
ds[dwgt.name] = dwgt
ds[dcrd.name] = dcrd

print('flux for each g')
ds

flux for each g


<xarray.Dataset>
Dimensions:   (g: 12, pressure: 75)
Coordinates:
  * pressure  (pressure) float64 0.0003 0.0008 0.0011 ... 953.6 977.3 1.001e+03
  * g         (g) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    WGT       (pressure, g) float64 0.03367 0.1235 0.4602 ... 2.388 7.304 71.04
    CRD       (pressure, g) float64 0.03367 0.1235 0.4602 ... 2.635 7.301 70.89

In [155]:
flux = pd.read_csv(runner.path/'fort.9', sep=r'\s+', skiprows=2, header=None)
flux.set_index(0, inplace=True)
flux.index.names = ['pressure']
flux.columns = ['CRD', 'WGT']

print('bands-total flux. crd and wgt.')
xr.Dataset.from_dataframe(flux)

bands-total flux. crd and wgt.


<xarray.Dataset>
Dimensions:   (pressure: 76)
Coordinates:
  * pressure  (pressure) float64 0.0 0.0006 0.0009 ... 965.5 989.2 1.013e+03
Data variables:
    CRD       (pressure) float64 114.5 114.5 114.5 114.5 ... 82.53 81.24 79.97
    WGT       (pressure) float64 114.5 114.5 114.5 114.5 ... 82.23 80.87 79.54

## LBLnewBestfitSWModelData

Class for managing the output data of lblnew-bestfit-sw

In [63]:
p = LBLnewBestfitSWParam(band=10, vmin=4400, vmax=8200, nv=10000, dv=0.001,
                     molecule='h2o', conc='atmpro',
                     ref_pts=[(300, 250)], ng_refs=[10], ng_adju=[0],
                     wgt=[(.95, .9, .45, .45, .45, .45, .5, .55, .7, .95)],
                     cosz=.2588, rsfc=0,
                     klin=5e-25,
                     option_k_lookup=0,
                     atmpro='trp',
                     commitnumber='79f2292')

runner = LBLnewBestfitSWRun(path=PATH_FORTRAN/p.to_path(), param=p)

In [64]:
datnames = {'flux':'output_flux.dat', 'heat':'output_heat.dat'}

In [71]:
pd.read_csv(runner.path/datnames['flux'], sep=r'\s+', 
            dtype={'level':np.int32, 'g':np.int32, 'pressure':np.float64, 'wgt_flux':np.float64, 'crd_flux':np.float64})

ValueError: could not convert string to float: '0.41554722-111'

In [56]:
df = pd.read_csv(runner.path/datnames['flux'], sep=r'\s+')
df.drop(['level'], axis=1, inplace=True)
df.set_index(['pressure', 'g'], inplace=True)
xr.Dataset.from_dataframe(df[['wgt_flux']])
xr.Dataset.from_dataframe(df[['crd_flux']])

df = pd.read_csv(runner.path/datnames['heat'], sep=r'\s+')
df.drop(['layer'], axis=1, inplace=True)
df.set_index(['pressure', 'g'], inplace=True)
xr.Dataset.from_dataframe(df[['wgt_heat']])
ds = xr.Dataset.from_dataframe(df[['crd_heat']])

ds#.rename({'crd_heat':'heatrg'})

<xarray.Dataset>
Dimensions:   (g: 10, pressure: 75)
Coordinates:
  * pressure  (pressure) float64 0.0003122 0.0007501 ... 977.3 1.001e+03
  * g         (g) int64 1 2 3 4 5 6 7 8 9 10
Data variables:
    crd_heat  (pressure, g) object '0.50732726E-01' ... '0.94231110E-01'

In [73]:
#export
class LBLnewBestfitSWModelData():
    def __init__(self, param, wgt_flux, wgt_heat, crd_flux, crd_heat):
        self.param = param
        self.wgt_flux, self.wgt_heat = wgt_flux, wgt_heat
        self.crd_flux, self.crd_heat = crd_flux, crd_heat

    @classmethod
    def from_runner(cls, runner):
        df = pd.read_csv(runner.path/'output_flux.dat', sep=r'\s+')
        df.drop(['level'], axis=1, inplace=True)
        df.set_index(['pressure', 'g'], inplace=True)
        wgt_flux = xr.Dataset.from_dataframe(df[['wgt_flux']]).rename({'wgt_flux':'fldg'})
        crd_flux = xr.Dataset.from_dataframe(df[['crd_flux']]).rename({'crd_flux':'fldg'})
        
        df = pd.read_csv(runner.path/'output_heat.dat', sep=r'\s+')
        df.drop(['layer'], axis=1, inplace=True)
        df.set_index(['pressure', 'g'], inplace=True)
        wgt_heat = xr.Dataset.from_dataframe(df[['wgt_heat']]).rename({'wgt_heat':'heatrg'})
        crd_heat = xr.Dataset.from_dataframe(df[['crd_heat']]).rename({'crd_heat':'heatrg'})
        return cls(runner.param, wgt_flux, wgt_heat, crd_flux, crd_heat)
         
    def from_mongodoc(self, doc):
        pass

In [74]:
md = LBLnewBestfitSWModelData.from_runner(runner)

In [59]:
md.wgt_flux

<xarray.Dataset>
Dimensions:   (g: 10, pressure: 76)
Coordinates:
  * pressure  (pressure) float64 0.0 0.0006244 0.0008759 ... 989.2 1.013e+03
  * g         (g) int64 1 2 3 4 5 6 7 8 9 10
Data variables:
    fldg      (pressure, g) object '0.52501509E-01' ... '0.28300473E+02'

In [60]:
md.crd_flux

<xarray.Dataset>
Dimensions:   (g: 10, pressure: 76)
Coordinates:
  * pressure  (pressure) float64 0.0 0.0006244 0.0008759 ... 989.2 1.013e+03
  * g         (g) int64 1 2 3 4 5 6 7 8 9 10
Data variables:
    fldg      (pressure, g) object '0.52501509E-01' ... '0.28550905E+02'

In [61]:
md.wgt_heat

<xarray.Dataset>
Dimensions:   (g: 10, pressure: 75)
Coordinates:
  * pressure  (pressure) float64 0.0003122 0.0007501 ... 977.3 1.001e+03
  * g         (g) int64 1 2 3 4 5 6 7 8 9 10
Data variables:
    heatrg    (pressure, g) object '0.48709283E-01' ... '0.11687957E+00'

In [62]:
md.crd_heat

<xarray.Dataset>
Dimensions:   (g: 10, pressure: 75)
Coordinates:
  * pressure  (pressure) float64 0.0003122 0.0007501 ... 977.3 1.001e+03
  * g         (g) int64 1 2 3 4 5 6 7 8 9 10
Data variables:
    heatrg    (pressure, g) object '0.50732726E-01' ... '0.94231110E-01'

# Export notebook

In [75]:
! python notebook2script.py 04_modeldata.ipynb

Converted 04_modeldata.ipynb to exp/nb_04.py


# fin

In [162]:
np.exp(-0.36), np.exp(-1.2)

(0.697676326071031, 0.30119421191220214)